# Build Agent
Agents are systems that use LLMs as reasoning engines to determine which actions to take and the inputs to pass them.

Concept
- Use language models tool calling ability
- Use search tool to lookup information from internet
- Compose LangGraph Agent to determine actions
- Debug with langsmith

In [ ]:
%pip install -U langchain-community langgraph langchain-anthropic tavily-python

In [2]:
# setup langsmith
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [3]:
# Tavily: (a search engine) as a tool
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [4]:
# using community tools
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1718483939, 'localtime': '2024-06-15 13:38'}, 'current': {'last_updated_epoch': 1718483400, 'last_updated': '2024-06-15 13:30', 'temp_c': 21.1, 'temp_f': 70.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 11.9, 'wind_kph': 19.1, 'wind_degree': 230, 'wind_dir': 'SW', 'pressure_mb': 1016.0, 'pressure_in': 30.01, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 46, 'cloud': 25, 'feelslike_c': 21.1, 'feelslike_f': 70.0, 'windchill_c': 17.3, 'windchill_f': 63.1, 'heatindex_c': 17.3, 'heatindex_f': 63.2, 'dewpoint_c': 7.9, 'dewpoint_f': 46.3, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 5.0, 'gust_mph': 13.9, 'gust_kph': 22.4}}"}, {'url': 'https://world-weather.info/forecast/u

In [5]:
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

In [21]:
# %pip install -qU langchain-openai
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4")
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [18]:
# %pip install langchain-experimental
# from langchain_experimental.llms.ollama_functions import OllamaFunctions
# from langchain_core.messages import HumanMessage

# model = OllamaFunctions(model="llama3", format="json")
# response = model.invoke([HumanMessage(content="hi!")])
# response.content

"Hi there! It's nice to meet you. How can I help you today?"

In [22]:
# In order to enable that we use .bind_tools to give the language model knowledge of these tools
model_with_tools = model.bind_tools(tools)

In [23]:
# Try1: let's first call with normal messages.
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [25]:
# Try2: let's now call with a message that should trigger the search tool.
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: content='' additional_kwargs={'tool_calls': [{'id': 'call_a3KglHoxCH65r10v9wNrjAcy', 'function': {'arguments': '{\n  "query": "current weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 88, 'total_tokens': 111}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run-d706fe92-f162-4401-bc33-798b64996bf6-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_a3KglHoxCH65r10v9wNrjAcy'}] usage_metadata={'input_tokens': 88, 'output_tokens': 23, 'total_tokens': 111}
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_a3KglHoxCH65r10v9wNrjAcy'}]


## create your own agent
Now that we have defined the tools and the LLM, we can create the agent. We will be using LangGraph to construct the agent.

Note that we are passing in the `model`, not `model_with_tools`. That is because `create_react_agent` will call .`bind_tools` for us under the hood.

In [31]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [27]:
# run agent
# these are all stateless queries (it won't remember previous interactions)
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='8be03176-d4fe-4648-9a74-9617109be51d'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-62216ea8-8be1-42ca-afeb-d240ca7c98c6-0', usage_metadata={'input_tokens': 83, 'output_tokens': 10, 'total_tokens': 93})]

In [28]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', id='3a2281e7-b229-4a6c-bc87-e24d3cc5739d'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_AXLUv01ZF0F9zLNGuA5Gqjdv', 'function': {'arguments': '{\n  "query": "current weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 88, 'total_tokens': 111}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a30889b6-ae02-4241-be9a-035eafd370c3-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_AXLUv01ZF0F9zLNGuA5Gqjdv'}], usage_metadata={'input_tokens': 88, 'output_tokens': 23, 'total_tokens': 111}),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America

## Streaming

In [32]:
# streaming messages
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ASkhAhmgLN43wfZlVxr5wqHO', 'function': {'arguments': '{\n  "query": "current weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 88, 'total_tokens': 111}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b2ef5a72-f5eb-45a7-a20b-2495a3d0f4a5-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_ASkhAhmgLN43wfZlVxr5wqHO'}], usage_metadata={'input_tokens': 88, 'output_tokens': 23, 'total_tokens': 111})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'t

In [33]:
# streaming tokens
# we can do this with the .astream_events method.
# Python 3.11 or higher.
async for event in agent_executor.astream_events(
  {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
  kind = event["event"]
  if kind == "on_chain_start":
    if (
      event["name"] == "Agent"
    ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
      print(
        f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
      )
  elif kind == "on_chain_end":
    if (
      event["name"] == "Agent"
    ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
      print()
      print("--")
      print(f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}")
  if kind == "on_chat_model_stream":
    content = event["data"]["chunk"].content
    if content:
      # Empty content in the context of OpenAI means
      # that the model is asking for a tool to be invoked.
      # So we only print non-empty content
      print(content, end="|")
  elif kind == "on_tool_start":
    print("--")
    print(f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}")
  elif kind == "on_tool_end":
    print(f"Done tool: {event['name']}")
    print(f"Tool output was: {event['data'].get('output')}")
    print("--")

/Users/poom/Desktop/learning/langchain/env/lib/python3.9/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


As| an| AI| language| model|,| I| don|'t| have| real|-time| capabilities| to| provide| current| weather| updates|.| However|,| you| can| check| the| weather| in| San| Francisco| by| using| a| weather| forecasting| website| or| app| like| the| National| Weather| Service|,| BBC| Weather|,| or| The| Weather| Channel|.| Alternatively|,| you| could| use| a| voice| assistant| device| like| Amazon|'s| Alexa|,| Google| Assistant|,| or| Apple|'s| Siri|,| which| can| provide| real|-time| weather| updates|.|

## Adding memory
Agent is stateless. This means it does not remember previous interactions. but we also have to pass in a thread_id when invoking the agent

In [34]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [35]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)
config = {"configurable": {"thread_id": "abc123"}}

In [36]:
# tell the name to agent memory
for chunk in agent_executor.stream(
  {"messages": [HumanMessage(content="hi im bob!")]}, config
):
  print(chunk)
  print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 85, 'total_tokens': 96}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-909d7efa-fd76-4f9f-9c3e-3b507b251d89-0', usage_metadata={'input_tokens': 85, 'output_tokens': 11, 'total_tokens': 96})]}}
----


In [37]:
# ask the name from agent memory
# but if you change the thread_id, it will not remember the name
for chunk in agent_executor.stream(
  {"messages": [HumanMessage(content="whats my name?")]}, config
):
  print(chunk)
  print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob.', response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 108, 'total_tokens': 114}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ca1493e5-505c-47b6-88a6-38fc8d4d69e4-0', usage_metadata={'input_tokens': 108, 'output_tokens': 6, 'total_tokens': 114})]}}
----
